## Исследование заемщиков



### Шаг 1. Откроем файл с данными и изучите общую информацию. 

In [7]:
import pandas as pd
data_score = pd.read_csv('/datasets/data.csv')
#data_score.info() # оценим данные
data_score.head() # взглянем на таблицу



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

Таблица требует предобработки данных. Присутствуют отрицательные значения стажа работы, необходимо систематизировать причины, есть пропуски данных. Для работы нам потребуются следующие столбы: children, dob_years, family_status, family_status_id, debt,	total_income, purpose 


### Шаг 2. Предобработка данных

### Обработка пропусков

In [8]:
data_score.isnull().sum() # оценим состояние с пропусками
 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Мы имеем пропуски в двух столбцах, заполним их средним знаечением подням(тк для ответа на поставленные задачи обработка отрицательных результатов в этом столбце не требуется) и медианным по зарплате.

In [9]:
data_score['days_employed'] = data_score['days_employed'].fillna(data_score['days_employed'].mean())
data_score['total_income'] = data_score['total_income'].fillna(data_score['total_income'].median())

In [10]:
data_score['children'] = data_score['children'].replace(20,2)  # заменим выбивающиеся значиния, предположив, что 20 это опечатка 2, а -1 ошибчго введено с 1
data_score['children'] = data_score['children'].replace(-1,1)
data_score['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [11]:
data_score = data_score.dropna()  # удалим все строки с пропусками

In [12]:
data_score.isnull().sum() # проверим результат

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [13]:
data_score['children'].value_counts() # проверим разноообразие значений, по детям есть вопросы 
                                      #к отрицательным значением, реальностью 20 детьми
    

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [14]:
data_score['family_status' ].value_counts() # Здесь все ок  debt, 


женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [15]:
data_score['family_status_id' ].value_counts() # все совпадает



0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [16]:
data_score['total_income' ].value_counts() #все ок

145017.937533    2175
112874.418757       1
104381.857170       1
182036.676828       1
122421.963500       1
                 ... 
133299.194693       1
115080.782380       1
84896.781597        1
153838.839212       1
150014.128510       1
Name: total_income, Length: 19351, dtype: int64

In [17]:
data_score['debt' ].value_counts()  # все ок

0    19784
1     1741
Name: debt, dtype: int64

In [18]:
data_score['purpose' ].value_counts()  #  требуется леммитизация

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

Мы удалили строки с пропускми данных, тк заменять их средними значениями считаю некорректно.
Некооретные данные по детям мы заменили, допустив, что это была техническая ошибка ввода.
По обоим случаям необходимо сообщить специалистам, ответственным за сбор информации.

### Замена типа данных

In [19]:
data_score.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Да у нас числовые типы стали object, т.к мы заменили пропуски на пробелы.

Чать индексов имеют тип даннных object, такие как days_employed, total_income. Удобнее будет привести их к цифровомоу значению.

In [20]:
data_score['days_employed'] = data_score['days_employed'].astype('int')
data_score['total_income'] = data_score['total_income'].astype('int')
data_score['education'] = data_score['education'].astype('str') 
data_score['purpose'] = data_score['purpose'].astype('str')
data_score.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

В таблице были не
корректные типы данных, то что измерется цифрами мы изменили на int64, категорийные данные оставили в формате object


### Обработка дубликатов

In [21]:
data_score['education' ].value_counts()
data_score.duplicated().sum()

54

Мы видим большое количество дубликатов, особенно из-за заполнения разными регистрами, приведем все к одному регистру

In [22]:

data_score['education' ] = data_score['education' ].str.lower()


In [23]:
data_score['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Таким образомы знаем ситуацию с уровнем образования у заемщиков, посмторим, что с семейным положеним и типом доходов:

In [24]:
data_score.duplicated().sum()

71

В таблице нет полностью дублированных строк.

In [25]:
data_score['purpose' ] = data_score['purpose' ].str.lower() # подготовимся к леммитизации
data_score['family_status' ] = data_score['family_status' ].str.lower()
data_score['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

### Лемматизация

In [27]:
from pymystem3 import Mystem
m = Mystem()
text = ' '.join(list(data_score['purpose']))
lemmas = m.lemmatize(text)
from collections import Counter
dict = Counter(lemmas)
dict

Counter({'покупка': 5912,
         ' ': 55201,
         'жилье': 4473,
         'приобретение': 462,
         'автомобиль': 4315,
         'дополнительный': 909,
         'образование': 4022,
         'сыграть': 774,
         'свадьба': 2348,
         'операция': 2610,
         'с': 2924,
         'на': 2233,
         'проведение': 777,
         'для': 1294,
         'семья': 641,
         'недвижимость': 6367,
         'коммерческий': 1315,
         'жилой': 1233,
         'строительство': 1881,
         'собственный': 635,
         'подержать': 858,
         'свой': 2235,
         'со': 630,
         'заниматься': 908,
         'сделка': 944,
         'получение': 1316,
         'высокий': 1375,
         'подержанный': 110,
         'профильный': 436,
         'сдача': 653,
         'ремонт': 612,
         '\n': 1})

In [28]:
dict_purpose = ['недвижимость','жилье','автомобиль','образование','свадьба','ремонт',]

def lemm_text(text):
    return m.lemmatize(text)
    
data_score['purpose'] = data_score['purpose'].apply(lemm_text)
data_score['purpose']

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21520                  [операция,  , с,  , жилье, \n]
21521               [сделка,  , с,  , автомобиль, \n]
21522                              [недвижимость, \n]
21523    [на,  , покупка,  , свой,  , автомобиль, \n]
21524             [на,  , покупка,  , автомобиль, \n]
Name: purpose, Length: 21525, dtype: object

In [29]:
def cat_purpose(purpose_lemmas):
    for purpose in dict_purpose:
        if purpose in purpose_lemmas:
            return purpose
        
data_score['purpose_category'] = data_score['purpose'].apply(cat_purpose)
data_score['purpose_category'].value_counts()

недвижимость    6367
жилье           4473
автомобиль      4315
образование     4022
свадьба         2348
Name: purpose_category, dtype: int64

### Категоризация данных

In [30]:
data_score['purpose_category'] = data_score['purpose_category'].replace('жилье','недвижимость')   # заменим и объединим значение недвижимость и жилье
data_score['purpose_category'].value_counts()

недвижимость    10840
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose_category, dtype: int64

In [31]:

data_score['total_income'].median() # Узнаем медиану по доходам


145017.0

In [32]:
data_score['total_income'].mean() # узнаем средний доход в таблице

165158.94276422763

In [33]:
data_score['total_income'].min() # минимальный
data_score['total_income'].min() # и максимальный

20667

In [34]:
def income_group(income):                                                             # Категоризируем ситуацию по доходам, посчитаем следующие группы от минимума до 70 тр (примерно половина медианы) 70 - 170, 170 и  больше
    
        if 0 < income < data_score['total_income'].median()/2:
                return 'средний'
        if income <= data_score['total_income'].mean():
                return 'большой'
        return 'свербольшой'
    
data_score['income_group'] = data_score['total_income'].apply(income_group)

data_score['income_group'].value_counts()


большой        12173
свербольшой     7691
средний         1661
Name: income_group, dtype: int64

### Вывод

Категоризировали ситуацию по доходам, создали группы:
от минимума до 70 тр (примерно половина медианы) - средние
70 - 170,                                        - большие     
170 и  больше                                    - сверхбольшие



### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [35]:
data_score['debt'].count()
allnumber = data_score['debt'].count() # количество всех строк с информацией о заемщиках
allnumber                              # 


21525

In [36]:
dept_0 = data_score[data_score['debt'] == 0]
dept_0.head(20)                              # проверил таблицу, со всеми строками без задолжностей
dept_1 = data_score[data_score['debt'] == 1]
dept_1.head(5   )                            # проверил таблицу с должниками

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,income_group
14,0,-1844,56,высшее,0,гражданский брак,1,F,компаньон,1,165127,"[покупка, , жилой, , недвижимость, \n]",недвижимость,большой
32,0,-4649,34,среднее,1,гражданский брак,1,F,сотрудник,1,139057,"[на, , проведение, , свадьба, \n]",свадьба,большой
38,0,-597,25,высшее,0,не женат / не замужем,4,M,сотрудник,1,192247,"[образование, \n]",образование,свербольшой
55,0,63046,54,среднее,1,гражданский брак,1,F,пенсионер,1,145017,"[сыграть, , свадьба, \n]",свадьба,большой
75,1,-2953,38,среднее,1,женат / замужем,0,M,сотрудник,1,81935,"[операция, , с, , недвижимость, \n]",недвижимость,большой


In [37]:
child_debt_0 = dept_0.groupby(['children']).agg({'debt':'count' }) # сделаем агг. таблицу по количеству клиентов с детьми среди недолжников
child_debt_1 = dept_1.groupby(['children']).agg({'debt':'count' }) # то же для должников

part_child = child_debt_1 / (child_debt_0 + child_debt_1)
part_child = part_child.dropna() # удалим пустые значения
#part_child = (part_child.astype('str') + '%')
part_child = part_child.sort_values(by ='debt', ascending = False)
part_child.style.format("{:.2%}")

,debt
children,
4,9.76%
2,9.48%
1,9.15%
3,8.18%
0,7.51%


### Вывод

Люди с детьми примерно на 2% чаще имеют задолжности, чем без детей

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [38]:
marriage_debt_0 = dept_0.groupby(['family_status']).agg({'debt':'count' }) # сделаем агг. таблицу по семейному положению
marriage_debt_1 = dept_1.groupby(['family_status']).agg({'debt':'count' }) # то же для должников

part_marriage = marriage_debt_1 / (marriage_debt_0 + marriage_debt_1)
part_marriage = part_marriage.dropna() # удалим пустые значения

part_marriage = part_marriage.sort_values(by ='debt', ascending = False)
part_marriage.style.format("{:.2%}")


,debt
family_status,
не женат / не замужем,9.74%
гражданский брак,9.29%
женат / замужем,7.52%
в разводе,7.11%
вдовец / вдова,6.56%


### Вывод

Таким образом мы видим, что каждый 10 незамужний/неженатый человек не имеет просрочку по кредиту,
это число снижается к людям с гражданским браком, затем примерно одинаковые 7% имеют женатые или разведенные,
и меньеше всего просрочек у вдоцов\вдов

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [39]:
income_debt_0 = dept_0.groupby(['income_group']).agg({'debt':'count' }) # сделаем агг. таблицу по уровню дохода
income_debt_1 = dept_1.groupby(['income_group']).agg({'debt':'count' }) # то же для должников

part_income = income_debt_1 / (income_debt_0 + income_debt_1)
part_income = part_income.dropna() # удалим пустые значения

part_income = part_income.sort_values(by ='debt', ascending = False)
part_income.style.format("{:.2%}")


,debt
income_group,
большой,8.45%
свербольшой,7.76%
средний,6.92%


### Вывод

Люди с большим уровнем дохода чаще всего имеют просрочку, но с очень небольшим отрывом, на втором месте сверхбольшие доходы и потом средние. Вывод  - люди с меньшим доходом дисциплинированней в ведении финансов.

- Как разные цели кредита влияют на его возврат в срок?

In [40]:
purpose_debt_0 = dept_0.groupby(['purpose_category']).agg({'debt':'count' }) # сделаем агг. таблицу по целям
purpose_debt_1 = dept_1.groupby(['purpose_category']).agg({'debt':'count' }) # то же для должников

part_purpose = purpose_debt_1 / (purpose_debt_0 + purpose_debt_1)
part_purpose = part_purpose.dropna() # удалим пустые значения

part_purpose = part_purpose.sort_values(by ='debt', ascending = False)
part_purpose.style.format("{:.2%}")

,debt
purpose_category,
автомобиль,9.34%
образование,9.20%
свадьба,7.92%
недвижимость,7.21%


Попробуем свести все в одну таблицу

In [ ]:
all_debt_0 = dept_0.groupby(['children','family_status','income_group','purpose_category']).agg({'debt':'count' }) # сделаем агг. таблицу по целям
all_debt_1 = dept_1.groupby(['children','family_status','income_group','purpose_category']).agg({'debt':'count' }) # то же для должников

part_all = all_debt_1 / (all_debt_0 + all_debt_1)
part_all = part_all.dropna() # удалим пустые значения

part_all = part_all.sort_values(by ='debt', ascending = False)
part_all.style.format("{:.2%}")


### Шаг 4. Общий вывод

При отдельном рассмотрении всех групп, мы видим что процент просрочек в каждой категории составляет approx 7-10%
Если составить общую таблицу и расмотреть частный случай, то мы видим другие цифры, где 75% людей с тремя детьми не отдают кредит на свадьбу, или половина людей с двумя и тремя детьми в 
разводе имеют просрочки по автокредиту. А люди в разводе со сверхбольшим доходом вероятностью 2.70% будут иметь просрочку по автокредиту, как и женатые люди с думя детьми и средним доходом только в  3.12% случаев.